# Converting Yelp Dataset json files to CSV

Medium Article - [Convert Yelp Dataset to CSV using Pandas](https://link.medium.com/0k0DEb3Qy1)

[GitHub Repo](https://github.com/gyhou/yelp_dataset)

## Data Source
https://www.yelp.com/dataset/
- Unzip the first .tar file, it will extract a single file
- Add .tar to the end of the file's name to unzip it again
- The second file will extract 6 json files

In [1]:
# Need to upgrade pandas to version 0.25 to use "explode" function
# !py -m venv venv
!venv\Scripts\activate
!pip install pandas
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   --- ------------------------------------ 0.8/8.7 MB 5.6 MB/s eta 0:00:02
   -------- ------------------------------- 1.8/8.7 MB 9.1 MB/s eta 0:00:01
   --------------- ------------------------ 3.4/8.7 MB 7.2 MB/s eta 0:00:01
   -------------------- ------------------- 4.5/8.7 MB 5.8 MB/s eta 0:00:01
   ---------------------- ----------------- 5.0/8.7 MB 5.9 MB/s eta 0:00:01
   -------------------------- ------------- 5.8/8.7 MB 4.9 MB/s eta 0:00:01
   --------------------------- ------------ 6.0/8.7 MB 4.6 MB/s eta 0:00:01
   --------------------------------- ------ 7.3/8.7 MB 4.7 MB/s eta 0:00:01
   ---------------------------------------  8.7/8.7 MB 4.8 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 4.7 MB/s  0:00:01
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   - ---------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
# import json
import sklearn
from sklearn.preprocessing import MultiLabelBinarizer
pd.__version__

'2.3.3'

### Load and Clean Business.json file

In [ ]:
# Set up your local path
business_json_path = 'business.json'
business = pd.read_json(business_json_path, lines=True)
print(business.shape)
business.head()


In [ ]:
# transformar em dict
import ast

def safe_to_dict(x):
    if isinstance(x, dict):
        return x
    if pd.isna(x):
        return {}
    try:
        return ast.literal_eval(x)
    except:
        return {}

In [ ]:
business['attributes'] = business['attributes'].apply(safe_to_dict)
business['hours'] = business['hours'].apply(safe_to_dict)

In [ ]:
# explode attributes
attr_df = business['attributes'].apply(pd.Series)
hours_df = business['hours'].apply(pd.Series)

In [ ]:
attr_df = attr_df.add_prefix("attr_")
hours_df = hours_df.add_prefix("hours_")

In [ ]:
business_2 = pd.concat([business.drop(['attributes', 'hours'], axis=1), attr_df, hours_df], axis=1)

In [ ]:
business_2["categories_list"] = business_2["categories"].fillna("").apply(lambda x: [c.strip() for c in x.split(",") if c.strip() != ""])

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
encoded = mlb.fit_transform(df["categories_list"])

cat_df = pd.DataFrame(encoded, columns=[f"cat_{c}" for c in mlb.classes_])
business_2 = pd.concat([business_2, cat_df], axis=1)

In [ ]:
business_2 = business_2.fillna("")

In [ ]:
# business.to_csv("yelp_business.csv", index=False) # Save as CSV file without the index column

In [ ]:
# There are significant amount of businesses that are not open anymore
business_2.is_open.value_counts()

is_open
1    119698
0     30648
Name: count, dtype: int64

In [ ]:
# Keep only business that are still open
# Drop columns that may not be relavent ?
df_business = business_2[business_2['is_open']==1].drop(['hours','is_open','review_count'], axis=1)
print(df_business.shape)
df_business.head()

(119698, 11)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,attributes,categories
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food"
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea..."
6,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,MO,63144,38.627695,-90.340465,2.5,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Sporting Goods, Fashion, Shoe Stores, Shopping..."
